# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [26]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [27]:
# Parameters
DATASET = "human_enhancers_ensembl_regulatory"


In [28]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_enhancers_ensembl_regulatory 0 64 10


# Data download

In [29]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

In [30]:
info(DATASET)

## TF Dataset object

In [31]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 171024 files belonging to 2 classes.


## Text vectorization

In [32]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

['', '[UNK]', 't', 'a', 'g', 'c', 'n']

In [33]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [34]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [35]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
2673/2673 [==============================] - 228s 85ms/step - loss: 0.6022 - binary_accuracy: 0.6707 - f1_score: 0.5326
Epoch 2/10
2673/2673 [==============================] - 215s 80ms/step - loss: 0.6007 - binary_accuracy: 0.6731 - f1_score: 0.5355
Epoch 3/10
2673/2673 [==============================] - 134s 50ms/step - loss: 0.5997 - binary_accuracy: 0.6731 - f1_score: 0.5371
Epoch 4/10
2673/2673 [==============================] - 134s 50ms/step - loss: 0.5991 - binary_accuracy: 0.6735 - f1_score: 0.5387
Epoch 5/10
2673/2673 [==============================] - 135s 50ms/step - loss: 0.5976 - binary_accuracy: 0.6738 - f1_score: 0.5407
Epoch 6/10
2673/2673 [==============================] - 133s 50ms/step - loss: 0.5968 - binary_accuracy: 0.6747 - f1_score: 0.5438
Epoch 7/10
2673/2673 [==============================] - 133s 50ms/step - loss: 0.5955 - binary_accuracy: 0.6755 - f1_score: 0.5472
Epoch 8/10
2673/2673 [==============================] - 133s 50ms/step - loss: 0.59

## Evaluation on the test set

In [36]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 42756 files belonging to 2 classes.


In [37]:
model.evaluate(test_ds)

669/669 [==============================] - 10s 15ms/step - loss: 0.6510 - binary_accuracy: 0.6304 - f1_score: 0.3425


[0.6510307192802429, 0.6304378509521484, 0.3424831032752991]